In [35]:
import pandas as pd
df = pd.read_csv('https://query.data.world/s/b44zo5ntiszic5qat6jjmuecniu4z2?dws=00000')[:10]

In [36]:
df.head()

,title,url,pid,formatted_url,brand,stock,f_assured,price,currency,original_price,...,formatted_specifications,avg_rating,reviews_count,category,sub_category_1,sub_category_2,breadcrumbs,payment_options,uniq_id,scraped_at
0,Bastex Back Cover for Moto G9 (Multicolor),https://www.flipkart.com/bastex-back-cover-mot...,itmb772e270a8476,https://www.flipkart.com/product/p/itmb772e270...,NaN,NaN,False,219,INR,999,...,Sales Package: 1 Mobile Cover | Model Number: ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3fff7830-b71e-5968-9f2a-af7f89378ad0,06/15/22
1,"Devilart Back Cover for Lava X11 (Multicolor,...",https://www.flipkart.com/devilart-back-cover-l...,itmfbqyhwqvedcdn,https://www.flipkart.com/product/p/itmfbqyhwqv...,NaN,NaN,False,188,INR,659,...,Sales Package: 1 Designer Case | Model Number:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0e6e67fb-df1d-51a6-a23d-8a1ddfe79dc0,06/15/22
2,SY Gifts Back Cover for Apple iPhone 7 (Multi...,https://www.flipkart.com/sy-gifts-back-cover-a...,itm99e6ac4c0b9cd,https://www.flipkart.com/product/p/itm99e6ac4c...,NaN,NaN,False,199,INR,599,...,Sales Package: 1 Mobile Back Cover | Model Num...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3f8a411a-2c26-5fea-a0ae-ee44dd75aec0,06/15/22
3,Exclusivebay Back Cover for Micromax Canvas 5 ...,https://www.flipkart.com/exclusivebay-back-cov...,itmafcf69ec9cf97,https://www.flipkart.com/product/p/itmafcf69ec...,NaN,"Hurry, Only 5 left!",True,219,INR,999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dc31f707-dea0-5a4d-bd7f-f7280fa253c9,06/15/22
4,Coolboys Back Cover for Tecno KE-6 / KE6 (Pur...,https://www.flipkart.com/coolboys-back-cover-t...,itm414c308bcbb71,https://www.flipkart.com/product/p/itm414c308b...,NaN,NaN,False,197,INR,659,...,Sales Package: 1 Designer soft Silicon Cover |...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265428fc-4ed3-5de8-b7f7-4e1e27231609,06/15/22


In [37]:
df.columns

Index(['title', 'url', 'pid', 'formatted_url', 'brand', 'stock', 'f_assured',
       'price', 'currency', 'original_price', 'discount', 'images', 'seller',
       'seller_rating', 'return_policy', 'description', 'highlights',
       'specifications', 'formatted_specifications', 'avg_rating',
       'reviews_count', 'category', 'sub_category_1', 'sub_category_2',
       'breadcrumbs', 'payment_options', 'uniq_id', 'scraped_at'],
      dtype='object')

In [38]:
df.fillna("0",inplace=True)
df.shape

(10, 28)

In [39]:
df.drop([ 'images', 'seller',
       'seller_rating', 'return_policy', 'description', 'highlights',
       'specifications', 'formatted_specifications', 'avg_rating',
       'reviews_count', 'category', 'sub_category_1', 'sub_category_2',
       'breadcrumbs', 'payment_options', 'uniq_id', 'scraped_at'], axis=1, inplace=True)

In [ ]:
df[:1]

,title,price,currency,original_price
0,Bastex Back Cover for Moto G9 (Multicolor),219,INR,999


In [40]:
def convert_to_sentences(row):
    sep = " __ "
    invalids = {"-", "/", "NA", "N\\A"}

    sents = []
    for k, v in row.items():

        if type(v) == list:
            if len(v) == 0: continue
            sent = k + " " + " ".join(v)

        elif type(v) == dict:
            if len(v) == 0: continue
            # sent = sep.join([f"{k2} {v2}" for k2, v2 in v.items() if v2 not in invalids])
            for k2, v2 in v.items():
                if v2 not in invalids:
                    sents.append(f"{k2} {v2}")
            continue

        else:
            if pd.isna(v): continue
            sent = f"{k} {v}"

        sent = sent.lower()
        sent = [i for i in sent.split(". ") if len(i) > 0]
        sents.extend(sent)

    sents = ". ".join(sents)
    sents = sents.lower()

    return sents

convert_to_sentences(df.iloc[0])

'title bastex back cover for moto g9\xa0\xa0(multicolor). url https://www.flipkart.com/bastex-back-cover-moto-g9/p/itmb772e270a8476. pid itmb772e270a8476. formatted_url https://www.flipkart.com/product/p/itmb772e270a8476. brand 0. stock 0. f_assured false. price 219. currency inr. original_price 999. discount 78% off'

In [41]:
df['sentences'] = df.apply(convert_to_sentences, axis=1)

In [8]:
!pip install qdrant-client[fastembed]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
!pip install --upgrade --quiet  lark qdrant-client langchain_community transformers accelerate bitsandbytes langchain_core
!pip install sentence_transformers
!pip install langchain
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 76.3 MB/s eta 0:00:00
ERROR: pip's depend

In [42]:
ids = df.index
documents = df['sentences']
# documents: List[str] = df['sentences']

In [43]:
metadata = df.fillna("").to_dict(orient='records')

In [44]:
# Name of Qdrant Collection for saving vectors
QD_COLLECTION_NAME = "ecomercedata"

In [45]:
import pandas as pd
from langchain.schema import Document
from typing import List

# Function to convert DataFrame to list of Documents
def df_to_documents(df: pd.DataFrame) -> List[Document]:
    documents = []
    for _, row in df.iterrows():
        metadata = {
            "title": row["title"],
            "url": row["url"],
            "pid": row["pid"],
            "formatted_url": row["formatted_url"],
            "brand": row["brand"],
            "stock": row["stock"],
            "f_assured": row["f_assured"],
            "price": row["price"],
            "currency": row["currency"],
            "original_price": row["original_price"],
            "discount": row["discount"],
            # "images": row["images"],
            # "seller": row["seller"],
            # "seller_rating": row["seller_rating"],
            # "return_policy": row["return_policy"],
            # "description": row["description"],
            # "highlights": row["highlights"],
            # "specifications": row["specifications"],
            # "avg_rating": row["avg_rating"],
            # "reviews_count": row["reviews_count"],
            # "category": row["category"],
            # "sub_category_1": row["sub_category_1"],
            # "sub_category_2": row["sub_category_2"],
            # "breadcrumbs": row["breadcrumbs"],
            # "payment_options": row["payment_options"],
            # "uniq_id": row["uniq_id"],
            # "scraped_at": row["scraped_at"]
        }
        doc = Document(page_content=row["sentences"], metadata=metadata)
        documents.append(doc)
    return documents

# Convert DataFrame to list of Documents
documents = df_to_documents(df)

# Print the first Document for verification
print(documents[0])


page_content='title bastex back cover for moto g9\xa0\xa0(multicolor). url https://www.flipkart.com/bastex-back-cover-moto-g9/p/itmb772e270a8476. pid itmb772e270a8476. formatted_url https://www.flipkart.com/product/p/itmb772e270a8476. brand 0. stock 0. f_assured false. price 219. currency inr. original_price 999. discount 78% off' metadata={'title': 'Bastex Back Cover for Moto G9\xa0\xa0(Multicolor)', 'url': 'https://www.flipkart.com/bastex-back-cover-moto-g9/p/itmb772e270a8476', 'pid': 'itmb772e270a8476', 'formatted_url': 'https://www.flipkart.com/product/p/itmb772e270a8476', 'brand': '0', 'stock': '0', 'f_assured': False, 'price': '219', 'currency': 'INR', 'original_price': '999', 'discount': '78% off'}


In [14]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [46]:
from langchain_community.vectorstores import Qdrant
from langchain_core.documents import Document

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name=QD_COLLECTION_NAME,
)

In [ ]:
# from qdrant_client import QdrantClient

# client = QdrantClient(":memory:")

# client.add(
#     collection_name=QD_COLLECTION_NAME,
#     ids=ids,
#     documents=documents,
#     metadata=metadata,
# )

# print("Completed")

In [ ]:
# # This function displays the results understandable output

# def display_results(results):
#     for i, result in enumerate(results):
#         print()
#         print(f"{i+1})")
#         for k, v in result.metadata.items():
#             if k != "document":
#                 print(f"{k.capitalize()}: {v}")

In [ ]:
# results = client.query(
#     collection_name=QD_COLLECTION_NAME,
#     query_text="amd ryzen laptops",
#     limit=5
# )
# display_results(results)

In [47]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
metadata_field_info = [
    AttributeInfo(name="title", description="Title of the product", type="string"),
    AttributeInfo(name="url", description="URL of the product page", type="string"),
    AttributeInfo(name="pid", description="Product ID", type="string"),
    AttributeInfo(name="formatted_url", description="Formatted URL of the product page", type="string"),
    AttributeInfo(name="brand", description="Brand of the product", type="string"),
    AttributeInfo(name="stock", description="Stock status of the product", type="string"),
    AttributeInfo(name="f_assured", description="Flipkart assured status", type="boolean"),
    AttributeInfo(name="price", description="Price of the product", type="float"),
    AttributeInfo(name="currency", description="Currency of the price", type="string"),
    AttributeInfo(name="original_price", description="Original price of the product", type="float"),
    AttributeInfo(name="discount", description="Discount on the product", type="string"),
    # AttributeInfo(name="images", description="Images of the product", type="string"),
    # AttributeInfo(name="seller", description="Seller of the product", type="string"),
    # AttributeInfo(name="seller_rating", description="Rating of the seller", type="float"),
    # AttributeInfo(name="return_policy", description="Return policy of the product", type="string"),
    # AttributeInfo(name="description", description="Description of the product", type="string"),
    # AttributeInfo(name="highlights", description="Highlights of the product", type="string"),
    # AttributeInfo(name="specifications", description="Specifications of the product", type="string"),
    # AttributeInfo(name="avg_rating", description="Average rating of the product", type="float"),
    # AttributeInfo(name="reviews_count", description="Number of reviews for the product", type="integer"),
    # AttributeInfo(name="category", description="Category of the product", type="string"),
    # AttributeInfo(name="sub_category_1", description="Sub-category level 1", type="string"),
    # AttributeInfo(name="sub_category_2", description="Sub-category level 2", type="string"),
    # AttributeInfo(name="breadcrumbs", description="Breadcrumbs for navigation", type="string"),
    # AttributeInfo(name="payment_options", description="Payment options available for the product", type="string"),
    # AttributeInfo(name="uniq_id", description="Unique ID for the product", type="string"),
    # AttributeInfo(name="scraped_at", description="Scraped date of the product", type="string"),
]

document_content_description = "Product details including title, price, brand, description, and specifications."

# Initialize LLM
# llm = ChatOpenAI(temperature=0)


In [17]:
!pip install bitsandbytes

In [48]:
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoConfig
from time import time
import transformers
import torch

In [50]:
model_checkpoint = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
model_config = AutoConfig.from_pretrained(model_checkpoint,
                                          trust_remote_code=True,
                                          max_new_tokens=1024)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [51]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint,
                                             trust_remote_code=True,
                                             config=model_config,
                                             device_map='auto')

In [52]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [55]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

In [56]:
pipeline = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    torch_dtype=torch.float16,
                    max_length=3000,
                    device_map="auto",)

In [57]:
llm = HuggingFacePipeline(pipeline=pipeline)

In [58]:
llm.invoke("I want redme info")

'I want redme info to be able to provide me with the best possible service.\n\nI appreciate your time and effort in helping me with my inquiry. Please let me know if you need any further information or if there is anything else I can do to assist you.\n\nThank you again for your time and attention to this matter.\n\nBest regards,\n\n[Your Name]'

In [59]:
vectorstore.similarity_search("I want redme info",k=1)

[Document(page_content='title quadronic back cover for redmi note 9 pro max, smoke translucent shock proof smooth rubberized matte hard back case cover\xa0\xa0(green). url https://www.flipkart.com/quadronic-back-cover-redmi-note-9-pro-max-smoke-translucent-shock-proof-smooth-rubberized-matte-hard-case/p/itm89ba80122e631. pid itm89ba80122e631. formatted_url https://www.flipkart.com/product/p/itm89ba80122e631. brand 0. stock hurry, only a few left!. f_assured true. price 179. currency inr. original_price 398. discount 55% off', metadata={'title': 'quadronic Back Cover for Redmi Note 9 Pro Max, Smoke Translucent Shock Proof Smooth Rubberized Matte Hard Back Case Cover\xa0\xa0(Green)', 'url': 'https://www.flipkart.com/quadronic-back-cover-redmi-note-9-pro-max-smoke-translucent-shock-proof-smooth-rubberized-matte-hard-case/p/itm89ba80122e631', 'pid': 'itm89ba80122e631', 'formatted_url': 'https://www.flipkart.com/product/p/itm89ba80122e631', 'brand': '0', 'stock': 'Hurry, Only a few left!', 

In [60]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
import lark

document_content_description = "Product details including title, price, brand, description, and specifications."


In [61]:
# Define allowed comparators list
allowed_comparators = [
    "$eq",  # Equal to (number, string, boolean)
    "$ne",  # Not equal to (number, string, boolean)
    "$gt",  # Greater than (number)
    "$gte",  # Greater than or equal to (number)
    "$lt",  # Less than (number)
    "$lte",  # Less than or equal to (number)
    "$in",  # In array (string or number)
    "$nin",  # Not in array (string or number)
    "$exists", # Has the specified metadata field (boolean)
]


In [62]:
from re import VERBOSE
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    k=1,VERBOSE=True
)

In [32]:
# This example only specifies a filter
retriever.get_relevant_documents("""i want redmi note 9 pro max""")

OutputParserException: Parsing text
Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
    "limit": int \ the number of documents to retrieve
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | lt | lte | gt | gte | like): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes t

In [63]:
# flake8: noqa
from langchain_core.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
PROMPT

In [64]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

In [66]:
print(prompt.format(query="i want redmi note 9 pro max"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [68]:
query_constructor.invoke(
    """Data Source:
```json
{
    "content": "Lyrics of a song",
    "attributes": {
        "artist": {
            "type": "string",
            "description": "Name of the song artist"
        },
        "length": {
            "type": "integer",
            "description": "Length of the song in seconds"
        },
        "genre": {
            "type": "string",
            "description": "The song genre, one of "pop", "rock" or "rap""
        }
    }
}
```

User Query:
What are songs that were not published on Spotify

Structured Request:
```json
{
    "query": "",
    "filter": "NO_FILTER"
}
```"""
)

OutputParserException: Parsing text
Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, state

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=ChromaTranslator(),
)

In [ ]:
retriever.invoke("i want redme info")